In [0]:
from pyspark.sql.functions import array, array_intersect, size, desc, abs, datediff, lit, when, col
from datetime import date

In [0]:
def recomendar_livros(titulo_livro, df_livros, numero_de_recomendacoes=5):

    livro_referencia = df_livros.filter(df_livros.title.contains((titulo_livro))).orderBy(desc("totalratings")).collect()[0]
    generos_referencia = array(*[lit(genero) for genero in livro_referencia.genre])
    autor_referencia = livro_referencia.author
    data_referencia = livro_referencia.publishedDate if not "Unknown" in str(livro_referencia.publishedDate) else date.today()
   
    recomendacoes = df_livros.withColumn('generos_em_comum', size(array_intersect(df_livros.genre, generos_referencia))) \
                            .filter(df_livros.title != titulo_livro) \
                            .filter('generos_em_comum > 0') \
                            .withColumn('diferenca_de_tempo', abs(datediff(df_livros.publishedDate, lit(data_referencia)))) \
                            .withColumn('pontuacao', df_livros.stars * df_livros.totalratings * 'generos_em_comum' / (1 + col("diferenca_de_tempo"))) \
                            .withColumn('autores_iguais', when(df_livros.author == autor_referencia, lit(1)).otherwise(lit(0))) \
                            .orderBy(desc("autores_iguais"), desc("pontuacao")) \
                            .select(col("title").alias('titulo'), col("author").alias('autor'), col("pages").alias('numero_de_paginas'), col("stars").alias('numero_de_estrelas'), col("totalratings").alias('total_de_avaliacoes'), col("price").alias('preco'), col("productURL").alias('url_do_produto')) \
                            .limit(numero_de_recomendacoes)

    return recomendacoes

In [0]:
df_goodreads = spark.read.format('delta').load('abfss://livros@recomendador.dfs.core.windows.net/silver/goodreads')
df_kindle = spark.read.format('delta').load('abfss://livros@recomendador.dfs.core.windows.net/silver/kindle')

In [0]:
df_livros = df_goodreads.join(df_kindle, df_goodreads.title == df_kindle.title, 'right') \
                    .drop(df_goodreads.title, df_goodreads.author, df_goodreads.rating, df_kindle.reviews)

df_livros = df_livros.repartition(100)

In [0]:
del df_goodreads, df_kindle

In [0]:
titulo_livro = input("Digite o título do livro para receber recomendação: ")
numero_de_recomendacoes = input("Digite quantas recomendações deseja ter: ")

resultado = recomendar_livros(titulo_livro, df_livros, int(numero_de_recomendacoes))

In [0]:
resultado.toPandas()